In [1]:
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool
import vertexai

# !pip install llama-index
from llama_index.core import PromptTemplate

from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

BUCKET_NAME = os.getenv("GCP_BUCKET_NAME")

In [3]:
# Initialize Vertex AI API once per session
project_id = !gcloud config get project
project_id = project_id.n
location = "us-central1"

vertexai.init(project=project_id, location=location)

# Create RagCorpus
display_name = "pii_corpus"
rag_corpus = rag.create_corpus(display_name=display_name)

In [4]:
# !pip install google-cloud-storage

from google.cloud import storage

directory_path = "pii_example_docs"

# Create a Storage client object
client = storage.Client()

# Get the bucket object
bucket = client.bucket(BUCKET_NAME)

# Get iterator of blobs with a prefix matching the directory path
blobs = bucket.list_blobs(prefix=directory_path)

In [5]:
# Create a RAG Corpus, Import Files, and Generate a response

paths = [f"gs://{BUCKET_NAME}/{blob.name}" for blob in blobs]  # Supports Google Cloud Storage and Google Drive Links

# Import Files to the RagCorpus
response = rag.import_files(
    rag_corpus.name,
    paths,
    chunk_size=512,  # Optional
    chunk_overlap=100,  # Optional
)

In [6]:
# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                    # Supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            similarity_top_k=3,  # Optional
            vector_distance_threshold=0.5,  # Optional
        ),
    )
)

In [7]:
# Create a gemini-pro model instance
MODEL_ID = "gemini-1.5-pro-preview-0514"
rag_model = GenerativeModel(
    model_name=MODEL_ID, tools=[rag_retrieval_tool]
)